In [ ]:
import numpy as np

In [ ]:
def player1(dice_total, my_dice, round_log):
	pass

def player2(dice_total, my_dice, round_log):
	pass

In [ ]:
players = [player1, player2]

In [ ]:
class Game:
	def __init__(self, players, dice_count) -> None:
		# setting up a rng generator
		self.rng = np.random.default_rng()

		# storing the players and the dice numbers, and shuffling the player orders
		self.game_overview = {"players": self.rng.permutation(players), "dice_count": [dice_count]*len(players)}

	
	def init_round(self):
		self.round_log = {"player": [], "dice_count": [], "amount": [], "face": []}
		self.round_data = {"total_dice": 0, "total_counts": [0]*6, "player_dice": []}


		for player in range(len(self.game_overview["players"])):
			player_dice = [0]*6
			# if self.game_overview["dice_count"][player] == 0:
			# 	self.round_data["player_dice"].append(player_dice)
			# 	continue

			for dice in range(self.game_overview["dice_count"][player]):
				value = self.rng.integers(1,7)
				self.round_data["total_dice"]+=1
				self.round_data["total_counts"][value-1] += 1
				player_dice[value-1] += 1
			print(player_dice)
			self.round_data["player_dice"].append(player_dice)

In [ ]:
game = Game(players, 5)
game.game_overview

In [ ]:
game.game_overview["dice_count"][0] = 0
game.game_overview

In [ ]:
game.init_round()
game.round_data